In [1]:
trade_map = { "I": "orig_id"
  ,
  "x":"exchange",

  "p":"price",
  
  
  "i":"id",
   
  "e":"correction",

  "r":"trf_id",

  "t":"sip_timestamp",

  "y":"participant_timestamp",

  "f":"trf_timestamp",

  "q":"sequence_number",

  "c":"conditions",

  "s":"size",

  "z":"tape"
 }

In [2]:
trade_conditions = {'0': 'Regular', '1': 'Acquisition', '2': 'AveragePrice', '3': 'AutomaticExecution', '4': 'Bunched', '5': 'BunchSold', '6': 'CAPElection', '7': 'CashTrade', '8': 'Closing', '9': 'Cross', '10': 'DerivativelyPriced', '11': 'Distribution', '12': 'FormT(ExtendedHours)', '13': 'FormTOutOfSequence', '14': 'InterMarketSweep', '15': 'MarketCenterOfficialClose', '16': 'MarketCenterOfficialOpen', '17': 'MarketCenterOpening', '18': 'MarketCenterReOpenning', '19': 'MarketCenterClosing', '20': 'NextDay', '21': 'PriceVariation', '22': 'PriorReferencePrice', '23': 'Rule155Amex', '24': 'Rule127Nyse', '25': 'Opening', '26': 'Opened', '27': 'RegularStoppedStock', '28': 'ReOpening', '29': 'Seller', '30': 'SoldLast', '31': 'SoldLastStoppedStock', '32': 'SoldOutOfSequence', '33': 'SoldOutOfSequenceStoppedStock', '34': 'Split', '35': 'StockOption', '36': 'YellowFlag', '37': 'OddLot', '38': 'CorrectedConsolidatedClosePrice', '39': 'Unknown', '40': 'Held', '41': 'TradeThruExempt', '42': 'NonEligible', '43': 'NonEligible-extended', '44': 'Cancelled', '45': 'Recovery', '46': 'Correction', '47': 'AsOf', '48': 'AsOfCorrection', '49': 'AsOfCancel', '50': 'OOB', '51': 'Summary', '52': 'Contingent', '53': 'Contingent(Qualified)', '54': 'Errored'}
quote_conditions = {'0': 'Regular', '1': 'RegularTwoSidedOpen', '2': 'RegularOneSidedOpen', '3': 'SlowAsk', '4': 'SlowBid', '5': 'SlowBidAsk', '6': 'SlowDueLRPBid', '7': 'SlowDueLRPAsk', '8': 'SlowDueNYSELRP', '9': 'SlowDueSetSlowListBidAsk', '10': 'ManualAskAutomaticBid', '11': 'ManualBidAutomaticAsk', '12': 'ManualBidAndAsk', '13': 'Opening', '14': 'Closing', '15': 'Closed', '16': 'Resume', '17': 'FastTrading', '18': 'TradingRangeIndication', '19': 'MarketMakerQuotesClosed', '20': 'NonFirm', '21': 'NewsDissemination', '22': 'OrderInflux', '23': 'OrderImbalance', '24': 'DueToRelatedSecurityNewsDissemination', '25': 'DueToRelatedSecurityNewsPending', '26': 'AdditionalInformation', '27': 'NewsPending', '28': 'AdditionalInformationDueToRelatedSecurity', '29': 'DueToRelatedSecurity', '30': 'InViewOfCommon', '31': 'EquipmentChangeover', '32': 'NoOpenNoResume', '33': 'SubPennyTrading', '34': 'AutomatedBidNoOfferNoBid', '35': 'LuldPriceBand', '36': 'MarketWideCircuitBreakerLevel1', '37': 'MarketWideCircuitBreakerLevel2', '38': 'MarketWideCircuitBreakerLevel3', '39': 'RepublishedLuldPriceBand', '40': 'OnDemandAuction', '41': 'CashOnlySettlement', '42': 'NextDaySettlement', '43': 'LULDTradingPause', '71': 'SlowDueLRPBidAsk'}

In [3]:
import datetime
import pandas as pd
import pytz
import time
eastern = 'US/Eastern'

from polygon import RESTClient

In [4]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_list_of_Exchanges():

    with RESTClient(key) as client:
        respex = client.stocks_equities_exchanges()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict

def get_list_of_trade_conditions():

    with RESTClient(key) as client:
        respex = client.stocks_equities_condition_mappings()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict
    
def update_condition_labels(keyFunction, values):
    #if type(list(values)) == list:
    k =  list(keyFunction[str(v)] for v in list(values))
    return ", ".join(k)
    #return values

In [5]:
def get_trades_all(ticker, date, size):
    total_data = []
    lasttimestamp = 0
    
    while True:
        with RESTClient(key) as client:
            
            if lasttimestamp == 0:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, limit=size, reverse=False)
            else:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, timestamp = lasttimestamp, limit=size, reverse=False)

            result = resp.__dict__
            
            if "results" in result:
                result = result['results']
            else:
                break
                
            lasttimestamp = result[-1]['t']
            total_data.extend(result)
            
            if len(result) == 50000:
                continue
            else:
                break
        
        time.sleep(1)
        
    return total_data

In [30]:
trades = get_trades_all("AAPL", "2021-02-17", 500)

In [31]:

trades_df = pd.DataFrame(trades).rename(columns=trade_map)
trades_df['sip_timestamp'] =pd.to_datetime(trades_df['sip_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)
trades_df['participant_timestamp'] =pd.to_datetime(trades_df['participant_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)

#### CHECK IF FULL PREMARKET DATA EXITS

In [32]:
trades_df['hours'] = trades_df['sip_timestamp'].apply(lambda x: x.hour)
trades_df['minutes'] = trades_df['sip_timestamp'].apply(lambda x: x.minute)
trades_df['seconds'] = trades_df['sip_timestamp'].apply(lambda x: x.second)

In [33]:
new_new_df = trades_df[(trades_df['hours'] == 9) & (trades_df['minutes'] > 28) & (trades_df['minutes'] < 31)]

## Check condition

In [34]:
trades_df.head(1)

,sip_timestamp,participant_timestamp,sequence_number,id,exchange,size,conditions,price,tape,hours,minutes,seconds
0,2021-02-17 04:00:00.126028972-05:00,2021-02-17 04:00:00.125662208-05:00,1490,1,11,1437,"[12, 41]",133.0,3,4,0,0


In [35]:
type(trades_df['conditions'][0])

list

In [36]:
for i in trades_df['conditions']:
    print(type(i))
    break

<class 'list'>


In [42]:
trades_df['conditions_string'] = trades_df['conditions'].apply(str)
trades_df[trades_df.conditions_string.str.contains("12")]

,sip_timestamp,participant_timestamp,sequence_number,id,exchange,size,conditions,price,tape,hours,minutes,seconds,conditions_string
0,2021-02-17 04:00:00.126028972-05:00,2021-02-17 04:00:00.125662208-05:00,1490,1,11,1437,"[12, 41]",133.00,3,4,0,0,"[12, 41]"
1,2021-02-17 04:00:01.314567223-05:00,2021-02-17 04:00:01.314200320-05:00,1523,2,11,7,"[12, 37]",132.99,3,4,0,1,"[12, 37]"
2,2021-02-17 04:00:01.314567393-05:00,2021-02-17 04:00:01.314200320-05:00,1524,3,11,33,"[12, 37]",132.98,3,4,0,1,"[12, 37]"
3,2021-02-17 04:00:05.992245508-05:00,2021-02-17 04:00:05.991880448-05:00,1616,4,11,1,"[12, 37]",132.81,3,4,0,5,"[12, 37]"
4,2021-02-17 04:00:06.751567822-05:00,2021-02-17 04:00:06.751196928-05:00,1634,5,11,40,"[12, 37]",133.00,3,4,0,6,"[12, 37]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-02-17 04:22:25.462136466-05:00,2021-02-17 04:22:25.462092956-05:00,12059,271,12,1,"[12, 37]",132.73,3,4,22,25,"[12, 37]"
496,2021-02-17 04:22:30.432772709-05:00,2021-02-17 04:22:30.432407552-05:00,12082,226,11,4,"[12, 37]",132.73,3,4,22,30,"[12, 37]"
497,2021-02-17 04:22:30.432772854-05:00,2021-02-17 04:22:30.432407552-05:00,12083,227,11,10,"[12, 37]",132.73,3,4,22,30,"[12, 37]"
498,2021-02-17 04:22:33.139186820-05:00,2021-02-17 04:22:33.139170981-05:00,12103,272,12,10,"[12, 37]",132.73,3,4,22,33,"[12, 37]"
